### Which country has produced the most successful tennis players?


In [3]:
import os
from dotenv import load_dotenv, find_dotenv
from tennis_data_analysis.db.mysql import Database
import pandas as pd

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)



True

## SQL Solution

#### find best country players base on number of wins
Top 10 country based on win

In [10]:
mysql = Database(os.environ.get('DB_HOST'), os.environ.get('DB_USERNAME'), os.environ.get('DB_PASSWORD'), os.environ.get('DB_DATABASE'), os.environ.get('DB_PORT'), True)

sql = """
SELECT pl.country, SUM(wins.num_of_win) AS country_wins
FROM (
    SELECT p.id AS player_id, p.full_name, p.slug, COUNT(r.winner_player_id) AS num_of_win
    FROM GameInfoResult AS r
    LEFT JOIN Players AS p ON p.id=r.winner_player_id
    WHERE r.winner_player_id != 0
    GROUP BY r.winner_player_id
    ORDER BY num_of_win DESC
) AS wins
JOIN Players AS pl ON pl.slug=wins.slug
GROUP BY pl.country
ORDER BY country_wins DESC
LIMIT 10
"""
mysql.db.execute(sql)
result = mysql.db.fetchall()

if result:
    for row in result:
        print(f"{row['country']}: {row['country_wins']}")
else:
    print("Not Found")


USA: 468
Italy: 453
France: 393
Japan: 318
Germany: 280
Russia: 267
Spain: 207
Argentina: 196
China: 195
Australia: 189


#### Find best country players base on average rank of players where count of country players is greater or equal than 50

~~~sql
SELECT p.country, AVG(current_rank) AS avg_rank
FROM Players AS p
WHERE p.current_rank > 0
GROUP BY p.country
HAVING COUNT(p.country) >= 50
ORDER BY avg_rank ASC
~~~

In [16]:
import tabulate as tabulate

sql = """
SELECT p.country, AVG(current_rank) AS avg_rank, COUNT(p.id) AS country_player_count
FROM Players AS p
WHERE p.current_rank > 0
GROUP BY p.country
HAVING country_player_count >= 50
ORDER BY avg_rank ASC
"""
mysql.db.execute(sql)
result = mysql.db.fetchall()

if result:
    result = [[i+1, row['country'], row['avg_rank'], row['country_player_count']] for i, row in enumerate(result)]
    print(tabulate.tabulate(result, headers=['#', 'Country', 'Rank Averag', 'Count of players'], tablefmt='outline', stralign="left", numalign="center"))
else:
    print("Not Found")


+-----+----------------+---------------+--------------------+
|  #  | Country        |  Rank Averag  |  Count of players  |
+=====+================+===============+====================+
|  1  | Australia      |    547.444    |         72         |
|  2  | Czech Republic |    568.833    |         60         |
|  3  | France         |    572.55     |        151         |
|  4  | Argentina      |    625.962    |         78         |
|  5  | Russia         |    639.958    |        118         |
|  6  | Spain          |    661.614    |         88         |
|  7  | USA            |    673.544    |        204         |
|  8  | Germany        |    702.645    |        107         |
|  9  | United Kingdom |    707.431    |         65         |
| 10  | China          |    711.915    |         71         |
| 11  | Japan          |    758.093    |        118         |
| 12  | Italy          |    781.258    |        178         |
| 13  | Romania        |    800.533    |         60         |
| 14  | 

## Pandas Solution

#### Make new parquet for result of each set

In [224]:
from tennis_data_analysis.config import RAW_DATA_DIR

# Make GameInfoSets.parquet and GameInfoResult.parquet

V1_DATA_RAW_DIR = RAW_DATA_DIR / '01'

game_info_df = pd.read_parquet(V1_DATA_RAW_DIR / "GameInfo.parquet")
game_info_df = game_info_df.groupby(['match_id', 'set_id'])[['home_score', 'away_score']].max()

df = pd.DataFrame([], columns=['match_id', 'set_id', 'home_score', 'away_score'])

for i, v in game_info_df.iterrows():
    s = pd.DataFrame({'match_id': [i[0]], 'set_id': [i[1]], 'home_score': [v['home_score']], 'away_score': [v['away_score']]})
    df = pd.concat([df, s], ignore_index=True)

def is_home_winner(x):
    if x['home_score'] >= x['away_score']:
        return 1
    return 0

def is_away_winner(x):
    if x['away_score'] >= x['home_score']:
        return 1
    return 0

df['home_win'] = df.apply(is_home_winner, axis=1)
df['away_win'] = df.apply(is_away_winner, axis=1)

df.to_parquet(V1_DATA_RAW_DIR / 'GameInfoSets.parquet')


#### Find final result for every match id and save to V1_DATA_RAW_DIR/GameInfoResult.parquet

In [341]:
df = pd.read_parquet(V1_DATA_RAW_DIR / 'GameInfoSets.parquet')
df = df.set_index('match_id')


df_home_team_info = pd.read_parquet(V1_DATA_RAW_DIR / 'MatchHomeTeamInfo.parquet')
df_away_team_info = pd.read_parquet(V1_DATA_RAW_DIR / 'MatchAwayTeamInfo.parquet')
df_tournaments = pd.read_parquet(V1_DATA_RAW_DIR / 'MatchTournamentInfo.parquet')
df_event = pd.read_parquet(V1_DATA_RAW_DIR / 'MatchEventInfo.parquet')
df_players = pd.read_parquet(V1_DATA_RAW_DIR / 'Players.parquet')


def find_winner(row):
    if row['home_win_count'] > row['away_win_count']:
        player = df_home_team_info[df_home_team_info['match_id'] == row['match_id']]
        return player.iloc[0]['slug'] if not player.empty else 'unknow'
    else:
        player = df_away_team_info[df_away_team_info['match_id'] == row['match_id']]
        return player.iloc[0]['slug'] if not player.empty else 'unknow'


def find_loser(row):
    if row['home_win_count'] < row['away_win_count']:
        player = df_home_team_info[df_home_team_info['match_id'] == row['match_id']]
        return player.iloc[0]['slug'] if not player.empty else 'unknow'
    else:
        player = df_away_team_info[df_away_team_info['match_id'] == row['match_id']]
        return player.iloc[0]['slug'] if not player.empty else 'unknow'


def find_first_to_serve(row):
    event = df_event[df_event['match_id'] == row['match_id']]
    return event.iloc[0]['first_to_serve'] if not event.empty else 0


def find_start_datetime(row):
    event = df_event[df_event['match_id'] == row['match_id']]
    return event.iloc[0]['start_datetime'] if not event.empty else 0


def find_tournament_id(row):
    event = df_tournaments[df_event['match_id'] == row['match_id']]
    return event.iloc[0]['tournament_id'] if not event.empty else 0


df['home_win_count'] = df.groupby('match_id', as_index=True)['home_win'].sum()
df['away_win_count'] = df.groupby('match_id', as_index=True)['away_win'].sum()


df = df.reset_index().drop_duplicates(subset='match_id', keep='last')[['match_id', 'home_win_count', 'away_win_count']]

df['winner_player_id'] = df.apply(find_winner, axis=1)
df['loser_player_id'] = df.apply(find_loser, axis=1)

df['first_to_serve'] = df.apply(find_first_to_serve, axis=1)
df['start_datetime'] = df.apply(find_start_datetime, axis=1)
df['tournament_id'] = df.apply(find_tournament_id, axis=1)

df = df.merge(df_players[['slug', 'country']].rename(columns={"country": "winner_country"}), left_on='winner_player_id', right_on='slug').drop(columns=['slug'])
df = df.merge(df_players[['slug', 'country']].rename(columns={"country": "loser_country"}), left_on='loser_player_id', right_on='slug').drop(columns=['slug'])
df.to_parquet(V1_DATA_RAW_DIR / 'GameInfoResult.parquet')

## Find winner countries from GameInfoResult.parquet

In [362]:
import pandas as pd
from tennis_data_analysis.config import RAW_DATA_DIR
V1_DATA_RAW_DIR = RAW_DATA_DIR / '01'
df = pd.read_parquet(V1_DATA_RAW_DIR / 'GameInfoResult.parquet')
df.groupby(['winner_country'])['winner_country'].count().sort_values(ascending=False)

winner_country
USA        410
Italy      356
France     342
Japan      257
Russia     228
          ... 
Burundi      1
Iran         1
Nigeria      1
Senegal      1
Syria        1
Name: winner_country, Length: 89, dtype: int64